# Скрипт для получения и парсинга статей с русской вики Warhammer 40k.
Извлекает:
- основной текст статьи (первый абзац),
- инфобокс и таблицы внутри него,
- ссылки на другие статьи,
- категории статьи.

Обрабатывает редиректы и возвращает структурированные данные для дальнейшей загрузки в базу данных или анализа.

In [2]:
import requests
from bs4 import BeautifulSoup

def fetch_article(title, max_redirects=10, redirect_chain=None):
    """Получаем статью, автоматически обрабатывая редиректы"""
    if redirect_chain is None:
        redirect_chain = []

    if max_redirects <= 0:
        print(f"⚠ Redirect limit reached for {title}")
        return None, redirect_chain

    base_url = "https://warhammer40k.fandom.com/ru/api.php"
    params = {
        "action": "parse",
        "page": title,
        "format": "json",
        "prop": "text|categories",
        "redirects": True
    }

    response = requests.get(base_url, params=params, timeout=15)
    response.raise_for_status()
    data = response.json()

    # Если MediaWiki сообщает об ошибке
    if "error" in data:
        print(f"⚠ Error fetching {title}: {data['error'].get('info')}")
        return None, redirect_chain

    # Проверка редиректов
    redirects_info = data.get("redirects")
    if redirects_info and len(redirects_info) > 0:
        new_title = redirects_info[0].get("to")
        if new_title and new_title != title:
            redirect_chain.append((title, new_title))
            return fetch_article(new_title, max_redirects-1, redirect_chain)

    return data, redirect_chain


def extract_first_paragraph(soup):
    """Возвращаем текст первого параграфа и ссылки внутри него"""
    for aside in soup.find_all("aside", {"class": "portable-infobox"}):
        aside.decompose()
    for figcaption in soup.find_all("figcaption"):
        figcaption.decompose()

    for p in soup.find_all("p"):
        txt = p.get_text(" ", strip=True)
        if not txt or len(txt) < 30:
            continue
        if "©" in txt or txt.lower().startswith("см.") or txt.endswith(":"):
            continue

        # Сохраняем ссылки из параграфа
        links = []
        for a in p.find_all("a", href=True):
            href = a["href"]
            if href.startswith("/ru/wiki/") and not any(x in href for x in [":", "#"]):
                if a.get("title"):
                    links.append(a["title"])

        return txt, list(set(links))

    return None, []


def parse_td(td, links):
    td_data = {}

    if td.has_attr("data-source"):
        td_data["Название колонки"] = td["data-source"]

    # все ссылки в td верхнего уровня
    for a in td.find_all("a", href=True):
        if a["href"].startswith("/ru/wiki/") and not any(x in a["href"] for x in [":", "#"]):
            if a.get("title"):
                links.append(a["title"])

    # вложенная таблица
    nested_table = td.find("table")
    if nested_table:
        td_data["Вложенная таблица"] = parse_table(nested_table, links)

    # содержимое <p>
    for p in td.find_all("p"):
        b = p.find("b")
        if b:
            label = b.get_text(" ", strip=True).replace(":", "")
            items, buf = [], ""
            for elem in p.children:
                if elem.name == "br":
                    if buf.strip() and buf.strip() not in [label, ":"]:
                        items.append(buf.strip())
                    buf = ""
                elif isinstance(elem, str):
                    buf += elem
                else:
                    buf += elem.get_text(" ", strip=True)
            if buf.strip() and buf.strip() not in [label, ":"]:
                items.append(buf.strip())
            td_data[label] = items if len(items) > 1 else items[0]
        else:
            text = p.get_text(" ", strip=True)
            if text:
                td_data.setdefault("Прочее", []).append(text)

    if not td_data:
        td_data["Текст"] = td.get_text(" ", strip=True)
    return td_data


def parse_table(table, links):
    table_data = []
    for row in table.find_all("tr"):
        row_data = {}
        tds = row.find_all("td")
        for td in tds:
            td_parsed = parse_td(td, links)
            col_name = td_parsed.pop("Название колонки", f"Колонка_{tds.index(td)}")
            row_data[col_name] = td_parsed
        if row_data:
            table_data.append(row_data)
    return table_data



def parse_table(table, links):
    table_data = []
    for row in table.find_all("tr"):
        row_data = {}
        tds = row.find_all("td")
        for td in tds:
            td_parsed = parse_td(td, links)
            col_name = td_parsed.pop("Название колонки", f"Колонка_{tds.index(td)}")
            row_data[col_name] = td_parsed
        if row_data:
            table_data.append(row_data)
    return table_data


def parse_infobox(soup):
    """Парсим инфобокс (включая таблицы) и извлекаем ссылки"""
    infobox = {}
    links = []
    aside = soup.find("aside", {"class": "portable-infobox"})
    if not aside:
        return infobox, links

    title_el = aside.find("h2", {"data-source": "Название"})
    if title_el:
        infobox["Название"] = title_el.get_text(" ", strip=True)
        for a in title_el.find_all("a", href=True):
            if a["href"].startswith("/ru/wiki/") and not any(x in a["href"] for x in [":", "#"]):
                if a.get("title"):
                    links.append(a["title"])

    for div in aside.select("div.pi-item.pi-data"):
        key_el = div.find("h3", class_="pi-data-label")
        val_el = div.find("div", class_="pi-data-value")
        if key_el and val_el:
            key = key_el.get_text(" ", strip=True)
            value = val_el.get_text(" ", strip=True)
            infobox[key] = value

            for a in val_el.find_all("a", href=True):
                if a["href"].startswith("/ru/wiki/") and not any(x in a["href"] for x in [":", "#"]):
                    if a.get("title"):
                        links.append(a["title"])

    # парсим таблицы в инфобоксе
    for table in aside.find_all("table"):
        caption = table.find("caption")
        key = caption.get_text(" ", strip=True) if caption else "Таблица"
        infobox[key] = parse_table(table, links)
    
        # Парсим названия сторон только из первой строки
        sides = {"Сторона1": "", "Сторона2": ""}
        first_row = table.find("tr")
        if first_row:
            for td in first_row.find_all("td"):
                if td.has_attr("data-source") and td["data-source"] in sides:
                    # Берём текст до первого <hr> или <p>
                    hr = td.find("hr")
                    p = td.find("p")
                    if hr:
                        # Получаем всё содержимое td до hr
                        elems = []
                        for elem in td.children:
                            if elem == hr:
                                break
                            if isinstance(elem, str):
                                elems.append(elem.strip())
                            else:
                                elems.append(elem.get_text(" ", strip=True))
                        text = " ".join(elems).strip()
                    elif p:
                        # Всё до первого <p>
                        elems = []
                        for elem in td.children:
                            if elem == p:
                                break
                            if isinstance(elem, str):
                                elems.append(elem.strip())
                            else:
                                elems.append(elem.get_text(" ", strip=True))
                        text = " ".join(elems).strip()
                    else:
                        # fallback — берём текст td до слова "Командование"
                        text = td.get_text(" ", strip=True).split("Командование")[0].strip()
    
                    sides[td["data-source"]] = text
    
        if any(sides.values()):
            infobox["Названия сторон"] = sides
    
        


    return infobox, list(set(links))



def parse_article(data):
    result = {}
    parse_data = data.get("parse", {})
    result["title"] = parse_data.get("title")
    categories = parse_data.get("categories", [])
    result["categories"] = [c["*"] for c in categories]
    html_content = parse_data.get("text", {}).get("*", "")
    soup = BeautifulSoup(html_content, "html.parser")

    infobox, infobox_links = parse_infobox(soup)
    first_paragraph, para_links = extract_first_paragraph(soup)

    result["infobox"] = infobox
    result["first_paragraph"] = first_paragraph
    result["links"] = list(set(infobox_links + para_links))  # объединяем ссылки

    return result


if __name__ == "__main__":
    article_title = "Битва за Боросские врата"
    data, redirects = fetch_article(article_title)
    #print(data)
    parsed = parse_article(data)

    print("Redirects:", redirects)
    print("Название:", parsed["title"])
    print("Категории:", parsed["categories"])
    print("Первый абзац:", parsed["first_paragraph"])
    print("Инфобокс:")
    for k, v in parsed["infobox"].items():
        print(f"  {k}: {v}")
    print("Ссылки:", parsed["links"])


Redirects: []
Название: Битва за Боросские врата
Категории: ['История', 'Сражения', 'Несущие_Слово', 'Ультрамарины', 'Серые_Рыцари']
Первый абзац: Битва за Боросские врата (англ. Battle of the Boros Gate ) — сражение за регион Боросские врата между силами Империума и космодесантниками Хаоса из Несущих Слово незадолго до начала 13-го Чёрного крестового похода .
Инфобокс:
  Название: Битва за Боросские врата
  Дата: ~999. М41
  Система: Борос
  Сегментум: Обскурус
  Итог: Силы Хаоса отброшены, но имперцы заплатили суровую цену за победу
  Стороны: [{'Сторона1': {}, 'Сторона2': {}}]
  Названия сторон: {'Сторона1': 'Хаос : Несущие Слово , культисты Хаоса', 'Сторона2': 'Империум Человечества : Белые Консулы , Ультрамарины , Серые Рыцари , Астра Милитарум'}
Ссылки: ['13-й Чёрный крестовый поход', 'Сегментум Обскурус', 'Имперская Гвардия', 'Ультрамарины', 'М41', 'Империум Человечества', 'Серые Рыцари', 'Белые Консулы', 'Хаос', 'Боросские врата', 'Культисты Хаоса', 'Несущие Слово']


In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import time
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

BASE_URL = "https://warhammer40k.fandom.com/ru/api.php"

def fetch_all_titles():
    titles = []
    params = {"action": "query", "list": "allpages", "aplimit": "max", "apfilterredir": "all", "format": "json"}
    while True:
        resp = requests.get(BASE_URL, params=params, timeout=15).json()
        pages = resp.get("query", {}).get("allpages", [])
        titles.extend([p["title"] for p in pages])
        if "continue" in resp:
            params.update(resp["continue"])
        else:
            break
    logger.info(f"Found {len(titles)} titles")
    return titles

In [1]:
import json
import time

if __name__ == "__main__":
    titles = fetch_all_titles()
    all_articles = []

    for i, article_title in enumerate(titles, 1):
        try:
            data, redirects = fetch_article(article_title)
            if data is None:  # ошибка уже обработана в fetch_article
                continue

            parsed = parse_article(data)

            article_record = {
                "original_title": article_title,
                "final_title": parsed["title"],
                "categories": parsed["categories"],
                "first_paragraph": parsed["first_paragraph"],
                "infobox": parsed["infobox"],
                "links": parsed["links"],
                "redirect_chain": redirects
            }

            all_articles.append(article_record)

            # печать прогресса
            print(f"[{i}/{len(titles)}] {article_title} → {parsed['title']} ✓")
            print("Категории:", parsed["categories"])
            print("Первый абзац:", parsed["first_paragraph"])
            print("Инфобокс:", parsed["infobox"])
            print("Ссылки:", parsed["links"])
            print("-----------------------------------")

        except Exception as e:
            print(f"⚠ Ошибка при обработке {article_title}: {e}")

        time.sleep(1)  # crawl-delay

    # Сохраняем весь результат в JSON файл
    with open("articlesfull.json", "w", encoding="utf-8") as f:
        json.dump(all_articles, f, ensure_ascii=False, indent=2)


NameError: name 'fetch_all_titles' is not defined

In [6]:
import json

def clean_details(details: dict) -> dict:
    """
    Убираем дублирующие заголовки вроде "Командование:" внутри списков.
    """
    cleaned = {}
    for key, value in details.items():
        if isinstance(value, list) and value:
            # если первый элемент совпадает с ключом + ":", убираем его
            first = str(value[0]).strip()
            if first.rstrip(":") == key:
                value = value[1:]
        if value:  # только если что-то осталось
            cleaned[key] = value
    return cleaned


def unify_parties(infobox: dict):
    """
    Унифицируем данные о сторонах в единую структуру:
    parties = [{"name": ..., "details": {...}}, ...]
    """
    parties = []

    # Если просто "Сторона": строка
    if "Сторона" in infobox and isinstance(infobox["Сторона"], str):
        parties.append({"name": infobox["Сторона"]})

    # Если есть "Названия сторон"
    names = infobox.get("Названия сторон", {})
    if not isinstance(names, dict):
        names = {}

    # Если есть "Стороны"
    sides = infobox.get("Стороны", [])
    if isinstance(sides, list) and sides:
        for side_dict in sides:
            for key, details in side_dict.items():
                name = names.get(key, key)  # если имя есть в Названиях сторон — берём его
                if isinstance(details, dict):
                    details = clean_details(details)
                party = {"name": name}
                if details:  # добавляем только если не пусто
                    party["details"] = details
                parties.append(party)
    elif names:  # если только "Названия сторон"
        for key, name in names.items():
            parties.append({"name": name})

    return parties if parties else None


if __name__ == "__main__":
    with open("articlesfull.json", "r", encoding="utf-8") as f:
        articles = json.load(f)

    updated_articles = []

    for article in articles:
        infobox = article.get("infobox", {})
        if isinstance(infobox, dict) and any(k in infobox for k in ["Сторона", "Стороны", "Названия сторон"]):
            unified = unify_parties(infobox)
            if unified:
                # Удаляем старые ключи
                for k in ["Сторона", "Стороны", "Названия сторон"]:
                    infobox.pop(k, None)
                # Добавляем унифицированные
                infobox["parties"] = unified

        updated_articles.append(article)

    # Сохраняем результат
    with open("articles_unified.json", "w", encoding="utf-8") as f:
        json.dump(updated_articles, f, ensure_ascii=False, indent=2)

    print("✅ Готово! Результат сохранён в 'articles_unified.json'")


✅ Готово! Результат сохранён в 'articles_unified.json'


# Работа с графом

## Заполнение графа

In [ ]:
NEO4J_AUTH = "neo4j"
NEO4J_PASS = "password"

In [48]:
import json
from neo4j import GraphDatabase
import re

# -------------------- Подключение --------------------
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

# -------------------- Очистка графа --------------------
def clear_graph(session):
    session.run("MATCH (n) DETACH DELETE n")
    print("🗑️ Граф очищен")

# -------------------- Вспомогательные функции --------------------
def normalize_category(cat):
    """Разбиваем категорию на текст до скобок и в скобках"""
    categories = []
    match = re.search(r'\((.*?)\)', cat)
    if match:
        inside = match.group(1).strip()
        if inside:
            categories.append(inside)
        outside = cat[:match.start()].strip()
        if outside:
            categories.append(outside)
    else:
        categories.append(cat.strip())
    return categories

def sanitize_rel_type(rel):
    """Превращаем строку в безопасный тип отношения для Neo4j, поддерживая кириллицу"""
    rel = rel.upper()
    # оставляем кириллицу, латиницу, цифры и _
    rel = re.sub(r'[^А-ЯЁA-Z0-9_]', '_', rel)
    rel = re.sub(r'_+', '_', rel)
    rel = rel.strip('_')
    if not rel:
        rel = "REL"
    return rel

def create_node(tx, title, types, properties):
    """Создает узел с title как уникальным идентификатором"""
    query = "MERGE (n {title: $title}) "
    if types:
        query += "SET " + ", ".join([f"n:`{t}`" for t in types]) + " "
    if properties:
        props_str = ", ".join([f"{k}: ${k}" for k in properties])
        query += f"SET n += {{{props_str}}}"
    tx.run(query, title=title, **properties)

def create_relation(tx, from_title, to_title, rel_type):
    """Создает связь между узлами с безопасным типом"""
    rel_type = sanitize_rel_type(rel_type)
    query = f"""
    MATCH (a {{title: $from_title}})
    MATCH (b {{title: $to_title}})
    MERGE (a)-[r:{rel_type}]->(b)
    """
    tx.run(query, from_title=from_title, to_title=to_title)

def process_parties(session, article_title, parties):
    """Обрабатываем parties с привязкой к конкретному сражению"""
    for party in parties:
        party_name = party.get("name")
        details = party.get("details", {})

        session.write_transaction(create_node, party_name, ["Сторона"], {})

        participation_node = f"{article_title} | {party_name}"
        session.write_transaction(create_node, participation_node, ["Участие_Стороны"], {})

        session.write_transaction(create_relation, article_title, participation_node, "УЧАСТВУЕТ")
        session.write_transaction(create_relation, participation_node, party_name, "ПРЕДСТАВЛЯЕТ")

        # Командование
        for commander in details.get("Командование", []):
            session.write_transaction(create_node, commander, ["Персонаж"], {})
            session.write_transaction(create_relation, participation_node, commander, "КОМАНДОВАНИЕ")

        # Все остальные ключи
        for key, value in details.items():
            if key == "Командование" or not value:
                continue

            # Превращаем строки в список из одного элемента
            items = value if isinstance(value, list) else [value]

            for item in items:
                session.write_transaction(create_node, item, [key], {})
                session.write_transaction(create_relation, participation_node, item, key)


def import_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)

    with driver.session() as session:
        clear_graph(session)
        for article in data:
            title = article.get("final_title") or article.get("original_title")
            first_paragraph = article.get("first_paragraph", "")
            categories = article.get("categories", [])
            infobox = article.get("infobox", {})

            types = list({t for cat in categories for t in normalize_category(cat) if t})
            session.write_transaction(create_node, title, types, {"first_paragraph": first_paragraph})

            for key, value in infobox.items():
                if key == "parties":
                    process_parties(session, title, value)
                    continue
                if not value:
                    continue
                rel_type = sanitize_rel_type(key)
                if isinstance(value, str):
                    session.write_transaction(create_node, value, [key], {})
                    session.write_transaction(create_relation, title, value, rel_type)
                elif isinstance(value, list):
                    for v in value:
                        if isinstance(v, str):
                            session.write_transaction(create_node, v, [key], {})
                            session.write_transaction(create_relation, title, v, rel_type)
                else:
                    print(f"⚠️ Пропущено сложное значение для {key}: {value}")

            for link in article.get("links", []):
                session.write_transaction(create_node, link, ["Статья"], {})
                session.write_transaction(create_relation, title, link, "ССЫЛКА")

if __name__ == "__main__":
    import_json("articles_unified.json")
    print("✅ Импорт завершен")


🗑️ Граф очищен


/tmp/ipykernel_5702/430324736.py:106: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, title, types, {"first_paragraph": first_paragraph})
/tmp/ipykernel_5702/430324736.py:116: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, value, [key], {})
/tmp/ipykernel_5702/430324736.py:117: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation, title, value, rel_type)
/tmp/ipykernel_5702/430324736.py:127: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, link, ["Статья"], {})
/tmp/ipykernel_5702/430324736.py:128: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation, title, link, "ССЫЛКА")
/tmp/ipykernel_5702/430324736.py:67: DeprecationWarning: write_transaction has been renamed to exe

✅ Импорт завершен


## Примеры запросов

In [14]:
import sqlite3
import logging
from neo4j import GraphDatabase

# Настройка логгера
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Подключение к Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

# Подключение к SQLite
sqlite_path = "warhammer_articles.db"

def add_participant_relationships():
    conn = sqlite3.connect(sqlite_path)
    cursor = conn.cursor()
    
    cursor.execute("SELECT final_title, entities FROM articles")
    rows = cursor.fetchall()
    
    with driver.session() as session:
        for battle_title, entities in rows:
            if not entities:
                continue
            
            entity_list = [e.strip() for e in entities.split(",") if e.strip()]
            if not entity_list:
                continue
            
            logger.info(f"⚔️ Обрабатываю сражение: {battle_title} (участников: {len(entity_list)})")
            
            found_entities = []
            not_found_entities = []
            
            for entity in entity_list:
                result = session.run("""
                MATCH (b:Сражения {title: $battle})
                OPTIONAL MATCH (p {title: $entity})
                WHERE p:Персонаж OR p:Персонажи_
                FOREACH (_ IN CASE WHEN p IS NULL THEN [] ELSE [1] END |
                    MERGE (b)-[:УЧАСТНИК]->(p)
                )
                RETURN p IS NOT NULL AS exists
                """, {"battle": battle_title, "entity": entity})

                
                record = result.single()
                if record and record["exists"]:
                    found_entities.append(entity)
                else:
                    not_found_entities.append(entity)
    
    conn.close()
    logger.info("🎯 Проставление связей 'УЧАСТНИК' завершено.")

if __name__ == "__main__":
    add_participant_relationships()


2025-09-19 22:23:34,310 [INFO] ⚔️ Обрабатываю сражение: 101-й полк Бета-Драконов (участников: 15)
2025-09-19 22:23:34,330 [INFO] ⚔️ Обрабатываю сражение: 110-й Кадианский полк (участников: 17)
2025-09-19 22:23:34,342 [INFO] ⚔️ Обрабатываю сражение: 13-й Чёрный крестовый поход (участников: 165)
2025-09-19 22:23:34,519 [INFO] ⚔️ Обрабатываю сражение: 13-й Штрафной легион (участников: 23)
2025-09-19 22:23:34,528 [INFO] ⚔️ Обрабатываю сражение: 13-я великая рота (участников: 55)
2025-09-19 22:23:34,550 [INFO] ⚔️ Обрабатываю сражение: 133-й полк Лямбда-Львов (участников: 12)
2025-09-19 22:23:34,561 [INFO] ⚔️ Обрабатываю сражение: 22-й полк Тета-Грифонов (участников: 18)
2025-09-19 22:23:34,571 [INFO] ⚔️ Обрабатываю сражение: 29-й полк Зета-Тигров (участников: 10)
2025-09-19 22:23:34,580 [INFO] ⚔️ Обрабатываю сражение: 32-й полк Тета-Орлов (участников: 7)
2025-09-19 22:23:34,584 [INFO] ⚔️ Обрабатываю сражение: 33-й полк Дельта-Фениксов (участников: 13)
2025-09-19 22:23:34,588 [INFO] ⚔️ Обраб

In [45]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def query_graph_depth(node_title, depth=2):
    """
    Показывает граф вокруг узла с заданной глубиной.
    depth=1 — ближайшие связи, depth=2 — связи через 1 промежуточный узел и т.д.
    """
    with driver.session() as session:
        result = session.run(
            f"""
            MATCH path=(n {{title: $title}})-[*1..{depth}]-(connected)
            RETURN nodes(path) AS nodes, relationships(path) AS rels
            """,
            title=node_title
        )

        printed = set()
        for record in result:
            nodes = record["nodes"]
            rels = record["rels"]
            for rel in rels:
                start = rel.start_node["title"]
                end = rel.end_node["title"]
                rtype = rel.type
                # избегаем дублирования
                key = (start, rtype, end)
                if key not in printed:
                    print(f"{start} -[{rtype}]-> {end}")
                    printed.add(key)

if __name__ == "__main__":
    query_graph_depth("ы", depth=1)


Первая война за Армагеддон | Хаос -[ВОЙСКА]-> ы
Первая война за Армагеддон | Империум Человечества -[ВОЙСКА]-> ы
Третья война за Армагеддон | Waaagh! Газгкулла -[ВОЙСКА]-> ы
Гареокская Прерогатива | Империум -[ПОТЕРИ]-> ы
Гареокская Прерогатива | Империум , Хаос -[ПОТЕРИ]-> ы
Ультрамарская кампания | Силы Хаоса -[ПОТЕРИ]-> ы
Ультрамарская кампания | Империум Человечества , эльдары -[ПОТЕРИ]-> ы


In [17]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def query_node_info_simple(node_title):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (n {title: $title})
            OPTIONAL MATCH (n)-[out_rel]->(out_node)
            OPTIONAL MATCH (in_node)-[in_rel]->(n)
            RETURN n.title AS title, n.first_paragraph AS text, labels(n) AS labels,
                   collect(DISTINCT {rel: type(out_rel), target: out_node.title, target_text: out_node.first_paragraph}) AS outgoing,
                   collect(DISTINCT {rel: type(in_rel), source: in_node.title, source_text: in_node.first_paragraph}) AS incoming
            """,
            title=node_title
        )

        for record in result:
            output = f"=== {record['title']}"
            if record.get("labels"):
                output += f" [{', '.join(record['labels'])}]"
            output += " ===\n"

            if record["text"]:
                output += f"Описание: {record['text']}\n"

            # Остальные исходящие связи
            others_out = [rel for rel in record["outgoing"] if rel["rel"] != "ССЫЛКА"]
            if others_out:
                output += "\n"
                for rel in others_out:
                    if rel["target"]:
                        output += f"  - {rel['rel']}: {rel['target']}\n"
                        if rel["target_text"]:
                            output += f"      {rel['target_text']}\n"

            # Входящие связи
            others_in = [rel for rel in record["incoming"] if rel["rel"] != "ССЫЛКА"]
            if others_in:
                output += "\n"
                for rel in others_in:
                    if rel["source"]:
                        output += f"  - {rel['rel']}: {rel['source']}\n"
                        if rel["source_text"]:
                            output += f"      {rel['source_text']}\n"

            return output

if __name__ == "__main__":
    info = query_node_info_simple("Абаддон")
    print(info)
    info = query_node_info_simple("Ультрамарская кампания")
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print(info)
    info = query_node_info_simple("Лев: Сын Леса (роман)")
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print(info)


=== Абаддон [Статья, Персонаж, Персонажи_, Сыны_Хоруса, Чёрный_Легион, Лунные_Волки] ===
Описание: Эзеки́ль Абаддо́н [ 1 ] (англ. Ezekyle Abaddon ), также известный как Абаддон Разоритель (англ. Abaddon the Despoiler ) — Воитель Хаоса и повелитель Чёрного Легиона , величайший чемпион Тёмных богов объединяющий всех космодесантников-предателей, а также других приспешников Губительных Сил против Империума Человечества .

  - РАСА: Человек
  - ФРАКЦИЯ: Космодесант Хаоса
      Космодесантники Хаоса (англ. Chaos Space Marines ), также известные как космодесантники-предатели , космодесантники-отступники или космодесантники-еретики , считаются одними из самых могущественных и привилегированных служителей Тёмных богов , а также самыми ненавистными врагами Империума Человечества .
  - ПРИНАДЛЕЖНОСТЬ: Чёрный Легион
      Чёрный Легион (англ. Black Legion ), ранее носивший имя Лунные Волки (англ. Luna Wolves ), а затем Сыны Хоруса (англ. Sons of Horus ) — XVI из двадцати первоначальных легионов ко

In [16]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def find_paths_exclude_race_links_unknown(node1, node2, max_length=5, limit=5):
    """
    Ищет несколько коротких путей между node1 и node2,
    игнорируя связи типа 'РАСА', 'ССЫЛКА', 'ФРАКЦИЯ', 'РОДНОЙ_МИР' 
    и не проходящие через узлы с title = 'неизвестно'.
    """
    with driver.session() as session:
        query = f"""
        MATCH p=(a {{title: $node1}})-[rels*..{max_length}]-(b {{title: $node2}})
        WHERE all(r IN rels 
                  WHERE type(r) <> 'РАСА' 
                    AND type(r) <> 'СТАТУС' 
                    AND type(r) <> 'ССЫЛКА'
                    AND type(r) <> 'ПРЕДСТАВЛЯЕТ'
                    AND type(r) <> 'ПОТЕРИ'
                    AND type(r) <> 'ВОЙСКА'
                    AND type(r) <> 'ПОГИБ')
          AND NONE(n IN nodes(p)[1..-1] WHERE 'Персонажи_' IN labels(n) OR n.title IN ['Неизвестно', 'Неизвестен'])
        RETURN [n IN nodes(p) | n.title] AS path,
               [r IN rels | type(r)] AS rels,
               length(p) AS len
        ORDER BY len ASC
        LIMIT $limit
        """
        result = session.run(query, node1=node1, node2=node2, limit=limit)
        outputs = []

        for record in result:
            path = record["path"]
            rels = record["rels"]
            output = ""
            for i in range(len(rels)):
                output += f"{path[i]} -[{rels[i]}]-> {path[i+1]}\n"
            outputs.append(output.strip())

        if outputs:
            return "\n\n".join(outputs)
        else:
            return "Пути не найдены"

if __name__ == "__main__":
    paths_info = find_paths_exclude_race_links_unknown("Ультрамарская кампания", "Абаддон", max_length=5, limit=15)
    print(paths_info)


Ультрамарская кампания -[СЕКТОР]-> Ультрамар
Ультрамар -[СЕКТОР]-> Калт
Калт -[СЕГМЕНТУМ]-> Ультима
Ультима -[СЕГМЕНТУМ]-> Падение Амонтепа II
Падение Амонтепа II -[УЧАСТНИК]-> Абаддон

Ультрамарская кампания -[СЕКТОР]-> Ультрамар
Ультрамар -[СЕКТОР]-> Подземная война
Подземная война -[СЕГМЕНТУМ]-> Ультима
Ультима -[СЕГМЕНТУМ]-> Падение Амонтепа II
Падение Амонтепа II -[УЧАСТНИК]-> Абаддон

Ультрамарская кампания -[СЕКТОР]-> Ультрамар
Ультрамар -[СЕКТОР]-> Оборона Соты: Плач Эгиды
Оборона Соты: Плач Эгиды -[СЕГМЕНТУМ]-> Ультима
Ультима -[СЕГМЕНТУМ]-> Падение Амонтепа II
Падение Амонтепа II -[УЧАСТНИК]-> Абаддон

Ультрамарская кампания -[СЕКТОР]-> Ультрамар
Ультрамар -[СЕКТОР]-> Нуцерия
Нуцерия -[СЕГМЕНТУМ]-> Ультима
Ультима -[СЕГМЕНТУМ]-> Падение Амонтепа II
Падение Амонтепа II -[УЧАСТНИК]-> Абаддон

Ультрамарская кампания -[СЕКТОР]-> Ультрамар
Ультрамар -[СЕКТОР]-> Битва за Арматуру
Битва за Арматуру -[СЕГМЕНТУМ]-> Ультима
Ультима -[СЕГМЕНТУМ]-> Падение Амонтепа II
Падение Амонтепа II

In [7]:
from neo4j import GraphDatabase
from itertools import combinations

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def find_paths_multiple_nodes(nodes: list, max_length=5, limit=5):
    """
    Ищет пути между всеми парами узлов из списка `nodes`,
    игнорируя связи типа 'РАСА', 'ССЫЛКА', 'ФРАКЦИЯ', 'РОДНОЙ_МИР'
    и не проходящие через узлы с title = 'Неизвестно'.
    """
    all_paths = {}
    
    with driver.session() as session:
        for node1, node2 in combinations(nodes, 2):
            query = f"""
            MATCH p=(a {{title: $node1}})-[rels*..{max_length}]-(b {{title: $node2}})
            WHERE all(r IN rels 
                      WHERE type(r) <> 'РАСА' 
                        AND type(r) <> 'СТАТУС' 
                        AND type(r) <> 'ССЫЛКА'
                        AND type(r) <> 'ПРЕДСТАВЛЯЕТ'
                        AND type(r) <> 'ПОТЕРИ'
                        AND type(r) <> 'ВОЙСКА'
                        AND type(r) <> 'ПОГИБ'
                        AND type(r) <> 'ДАТА')
              AND NONE(n IN nodes(p)[1..-1] WHERE 'Персонажи_' IN labels(n) OR n.title IN ['Неизвестно', 'Неизвестен'])
            RETURN [n IN nodes(p) | n.title] AS path,
                   [r IN rels | type(r)] AS rels,
                   length(p) AS len
            ORDER BY len ASC
            LIMIT $limit
            """
            result = session.run(query, node1=node1, node2=node2, limit=limit)
            outputs = []

            for record in result:
                path = record["path"]
                rels = record["rels"]
                output = ""
                for i in range(len(rels)):
                    output += f"{path[i]} -[{rels[i]}]-> {path[i+1]}\n"
                outputs.append(output.strip())

            all_paths[f"{node1} ↔ {node2}"] = outputs if outputs else ["Пути не найдены"]
    
    return all_paths

# Пример использования
if __name__ == "__main__":
    nodes_list = ["Тразин", "Абаддон", "Робаут Жиллиман"]
    paths_info = find_paths_multiple_nodes(nodes_list, max_length=5, limit=10)
    
    for pair, paths in paths_info.items():
        print(f"=== Пути между {pair} ===")
        print("\n\n".join(paths))
        print("\n" + "="*50 + "\n")


=== Пути между Тразин ↔ Абаддон ===
Тразин -[УЧАСТНИК]-> 13-й Чёрный крестовый поход
13-й Чёрный крестовый поход -[УЧАСТНИК]-> Абаддон

Тразин -[КОМАНДОВАНИЕ]-> 13-й Чёрный крестовый поход | Империум Человечества при поддержке эльдар и некронов
13-й Чёрный крестовый поход | Империум Человечества при поддержке эльдар и некронов -[УЧАСТВУЕТ]-> 13-й Чёрный крестовый поход
13-й Чёрный крестовый поход -[УЧАСТНИК]-> Абаддон

Тразин -[УЧАСТНИК]-> 13-й Чёрный крестовый поход
13-й Чёрный крестовый поход -[НАЗВАНИЕ]-> 13-й Чёрный крестовый поход
13-й Чёрный крестовый поход -[УЧАСТНИК]-> Абаддон

Тразин -[УЧАСТНИК]-> 13-й Чёрный крестовый поход
13-й Чёрный крестовый поход -[УЧАСТВУЕТ]-> 13-й Чёрный крестовый поход | Силы Хаоса
13-й Чёрный крестовый поход | Силы Хаоса -[КОМАНДОВАНИЕ]-> Абаддон

Тразин -[КОМАНДОВАНИЕ]-> 13-й Чёрный крестовый поход | Империум Человечества при поддержке эльдар и некронов
13-й Чёрный крестовый поход | Империум Человечества при поддержке эльдар и некронов -[УЧАСТВУЕТ]-

# Вычисление связанности узлов

In [22]:
from neo4j import GraphDatabase
from itertools import combinations

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def calculate_graph_scores(nodes: list, max_length=5):
    """
    Для каждого узла считает графовый скор по отношению к остальным узлам:
    score = sum(1 / (1 + длина кратчайшего пути до каждого другого узла))
    """
    node_scores = {node: 0.0 for node in nodes}
    
    with driver.session() as session:
        for node1, node2 in combinations(nodes, 2):
            query = f"""
            MATCH p=(a {{title: $node1}})-[rels*..{max_length}]-(b {{title: $node2}})
            WHERE all(r IN rels 
                      WHERE type(r) <> 'РАСА' 
                        AND type(r) <> 'СТАТУС' 
                        AND type(r) <> 'ССЫЛКА'
                        AND type(r) <> 'ПРЕДСТАВЛЯЕТ'
                        AND type(r) <> 'ПОТЕРИ'
                        AND type(r) <> 'ВОЙСКА'
                        AND type(r) <> 'ПОГИБ')
              AND NONE(n IN nodes(p)[1..-1] WHERE 'Персонажи_' IN labels(n) OR n.title IN ['Неизвестно', 'Неизвестен'])
            RETURN length(p) AS path_length
            ORDER BY path_length ASC
            LIMIT 1
            """
            result = session.run(query, node1=node1, node2=node2)
            record = result.single()
            if record and record["path_length"] is not None:
                dist = record["path_length"]
                score = 1 / (1 + dist)
                # добавляем score обоим узлам
                node_scores[node1] += score
                node_scores[node2] += score
            else:
                # если пути нет — добавляем 0
                node_scores[node1] += 0
                node_scores[node2] += 0
    
    return node_scores

if __name__ == "__main__":
    nodes_list = ["Ультрамарская кампания", "Имперский Культ", "Робаут Жиллиман", "Язык орков", "Неодолимый крестовый поход", "Терранский крестовый поход", "Психическое пробуждение", "Принцепс"]
    #nodes_list = ["Абаддон", "Робаут Жиллиман", "Тразин"]
    scores = calculate_graph_scores(nodes_list)
    
    print("=== Графовые скоры узлов ===")
    for node, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
        print(f"{node}: {score:.3f}")


=== Графовые скоры узлов ===
Робаут Жиллиман: 1.500
Неодолимый крестовый поход: 0.833
Терранский крестовый поход: 0.833
Ультрамарская кампания: 0.500
Имперский Культ: 0.000
Язык орков: 0.000
Психическое пробуждение: 0.000
Принцепс: 0.000


In [15]:
from neo4j import GraphDatabase
from itertools import combinations

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def describe_node_connections(nodes: list, max_length=5):
    """
    Для каждого узла возвращает словарь с кратчайшими связями к другим узлам.
    Выводит только найденные пути.
    """
    connections = {node: {} for node in nodes}
    
    with driver.session() as session:
        for node1, node2 in combinations(nodes, 2):
            query = f"""
            MATCH p=(a {{title: $node1}})-[rels*..{max_length}]-(b {{title: $node2}})
            WHERE all(r IN rels 
                      WHERE type(r) <> 'РАСА' 
                        AND type(r) <> 'СТАТУС' 
                        AND type(r) <> 'ССЫЛКА'
                        AND type(r) <> 'ПРЕДСТАВЛЯЕТ'
                        AND type(r) <> 'ПОТЕРИ'
                        AND type(r) <> 'ВОЙСКА'
                        AND type(r) <> 'ПОГИБ'
                        AND type(r) <> 'ДАТА'
                        AND type(r) <> 'СЕГМЕНТУМ'
                        AND type(r) <> 'СЕКТОР'
                        AND type(r) <> 'ЖАНР'
                        AND type(r) <> 'ПРЕДЫДУЩАЯ'
                        AND type(r) <> 'ИЗДАТЕЛЬ'
                        AND type(r) <> 'СЛЕДУЮЩАЯ'
                        AND type(r) <> 'ПРИНАДЛЕЖНОСТЬ')
              AND NONE(n IN nodes(p)[1..-1] WHERE 'Персонажи_' IN labels(n) OR n.title IN ['Неизвестно', 'Неизвестен'])
            RETURN [n IN nodes(p) | n.title] AS path,
                   length(p) AS length
            ORDER BY length ASC
            LIMIT 1
            """
            record = session.run(query, node1=node1, node2=node2).single()
            if record:
                path = record["path"]
                connections[node1][node2] = path
                connections[node2][node1] = path  # симметрично
    
    return connections

if __name__ == "__main__":
    nodes_list = [
    "Робаут Жиллиман",
    "Катарина Грейфакс",
    "Сыны Жиллимана",
    "Ультрамарская кампания",
    "Неодолимый крестовый поход",
    "Орден Генезис",
    "Хранители Книг",
    "Абаддон",
    "Покорение Праксила"
]


    connections = describe_node_connections(nodes_list)

    for node, links in connections.items():
        if not links:
            continue  # пропускаем узлы без связей
        for target, path in links.items():
            path_str = " -> ".join(path)
            print(f"{path_str}")
        print()


Робаут Жиллиман -> Терранский крестовый поход -> Катарина Грейфакс
Робаут Жиллиман -> Ультрамарины -> Сыны Жиллимана
Робаут Жиллиман -> Ультрамарская кампания
Робаут Жиллиман -> Неодолимый крестовый поход
Робаут Жиллиман -> Ультрамарины -> Орден Генезис
Робаут Жиллиман -> Ультрамарины -> Хранители Книг
Робаут Жиллиман -> Улланорский крестовый поход -> Абаддон
Робаут Жиллиман -> Покорение Праксила

Робаут Жиллиман -> Терранский крестовый поход -> Катарина Грейфакс
Катарина Грейфакс -> Ультрамарская кампания
Катарина Грейфакс -> Терранский крестовый поход -> Адептус Арбитрес -> Неодолимый крестовый поход
Катарина Грейфакс -> 13-й Чёрный крестовый поход -> Абаддон

Робаут Жиллиман -> Ультрамарины -> Сыны Жиллимана
Сыны Жиллимана -> Ультрамарины -> Орден Генезис
Сыны Жиллимана -> Ультрамарины -> Хранители Книг

Робаут Жиллиман -> Ультрамарская кампания
Катарина Грейфакс -> Ультрамарская кампания

Робаут Жиллиман -> Неодолимый крестовый поход
Катарина Грейфакс -> Терранский крестовый поход 

In [27]:
from neo4j import GraphDatabase
from itertools import combinations

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO4J_AUTH, NEO4J_PASS))

def calculate_graph_metrics(nodes: list, max_length=5):
    """
    Возвращает:
    1) node_scores: {узел: графовый скор}
    2) paths_between_nodes: {(node1, node2): путь с типами связей}
    3) intermediate_nodes: список промежуточных узлов, которых нет в nodes
    """
    node_scores = {node: 0.0 for node in nodes}
    paths_between_nodes = {}
    intermediate_nodes = set()

    with driver.session() as session:
        for node1, node2 in combinations(nodes, 2):
            query = f"""
            MATCH p=(a {{title: $node1}})-[rels*..{max_length}]-(b {{title: $node2}})
            WHERE all(r IN rels 
                      WHERE type(r) <> 'РАСА' 
                        AND type(r) <> 'СТАТУС' 
                        AND type(r) <> 'ССЫЛКА')
              AND NONE(n IN nodes(p)[1..-1] WHERE 
                    ANY(l IN labels(n) WHERE l IN ['Персонажи_', 'Организации_Империума'])
                    OR n.title IN ['Неизвестно', 'Неизвестен']
                )
            RETURN [n IN nodes(p) | n.title] AS path,
                   [r IN relationships(p) | type(r)] AS rels,
                   length(p) AS path_length
            ORDER BY path_length ASC
            LIMIT 1
            """
            record = session.run(query, node1=node1, node2=node2).single()
            if record and record["path_length"] is not None:
                dist = record["path_length"]
                score = 1 / (1 + dist)
                node_scores[node1] += score
                node_scores[node2] += score

                path_nodes = record["path"]
                path_rels = record["rels"]

                # Формируем путь с типами связей
                path_with_rels = []
                for i in range(len(path_nodes) - 1):
                    path_with_rels.append(path_nodes[i])
                    path_with_rels.append(f"-[{path_rels[i]}]->")
                path_with_rels.append(path_nodes[-1])

                paths_between_nodes[(node1, node2)] = path_with_rels
                paths_between_nodes[(node2, node1)] = path_with_rels  # симметрично

                # Добавляем промежуточные узлы
                for n in path_nodes[1:-1]:
                    if n not in nodes:
                        intermediate_nodes.add(n)
            else:
                # путь не найден
                node_scores[node1] += 0
                node_scores[node2] += 0

    return node_scores, paths_between_nodes, list(intermediate_nodes)


# Пример использования
if __name__ == "__main__":
    nodes_list = [
        "Кадия",
        "Кадианские пилоны",
    ]

    scores, paths, intermediates = calculate_graph_metrics(nodes_list)

    print("Graph scores:")
    for node, score in scores.items():
        print(f"{node}: {score:.3f}")

    print("\nIntermediate nodes (not in original list):")
    print(intermediates)

    print("\nPaths with relationship types:")
    for key, path in paths.items():
        print(f"{' '.join(path)}")


Graph scores:
Кадия: 0.000
Кадианские пилоны: 0.000

Intermediate nodes (not in original list):
[]

Paths with relationship types:
